In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import json
import random
from skimage.segmentation import mark_boundaries
import warnings
warnings.filterwarnings('ignore')
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=7b57e7e59011514165436dd09ad0ef5cfc799895a167ae4b3db27c1e529d50ce
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
# Step 1: Load the JSON file and handle multiple JSON objects
def load_json_data(json_path, limit=10000):
    data = []
    with open(json_path, 'r') as f:
        for line in f:  # Read file line by line in case it's not an array
            try:
                data.append(json.loads(line))  # Load each JSON object line by line
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")

    # Shuffle the data and take a subset of size 'limit'
    random.shuffle(data)
    data = data[:limit]  # Select only the first 'limit' samples
    return data

In [ ]:
json_file_path = 'photos.json'  # Update with the correct path
json_data = load_json_data(json_file_path, limit=10000)  # Limit to 10,000 samples

In [ ]:
# Step 3: Load Images from JSON metadata and apply preprocessing

def load_images_with_preprocessing(image_dir, json_data, img_size=(224, 224)):
    images = []
    labels = []

    for item in json_data:
        photo_id = item['photo_id']
        label = item['label']

        image_path = os.path.join(image_dir, f"{photo_id}.jpg")
        if os.path.exists(image_path):
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)

            if image is None:
                print(f"Warning: Failed to load image {image_path}. Skipping.")
                continue

            try:
                image = cv2.resize(image, img_size)
            except Exception as e:
                print(f"Error resizing image {image_path}: {e}")
                continue

            image = detailed_preprocessing(image)
            image = image.astype('float32') / 255.0

            images.append(image)
            labels.append(label)
        else:
            print(f"Image {image_path} not found.")

    images = np.array(images)
    labels = np.array(labels)

    if images.ndim == 3:
        images = np.expand_dims(images, axis=-1)

    return images, labels

In [ ]:
# Directory where the photos are stored
image_dir = r'"C:\Users\LACHU\Downloads\yelp_photos\photos"'

# Load images and labels from the JSON data
images, labels = load_images_with_preprocessing(image_dir, json_data)

# Load images and labels from the JSON data
images, labels = load_images_with_preprocessing(image_dir, json_data)

Streaming output truncated to the last 5000 lines.
Image "C:\Users\LACHU\Downloads\yelp_photos\photos"/4mW6KeANvD217bqVhTpnxg.jpg not found.
Image "C:\Users\LACHU\Downloads\yelp_photos\photos"/OG5puzbrQuSP_HNQA8RWqg.jpg not found.
Image "C:\Users\LACHU\Downloads\yelp_photos\photos"/HBCc3ID0xDtJRgKCULpe7w.jpg not found.
Image "C:\Users\LACHU\Downloads\yelp_photos\photos"/3BeJ8jwObcSstFIfM_WZSg.jpg not found.
Image "C:\Users\LACHU\Downloads\yelp_photos\photos"/Uvkf6LcTd9xptcbfOi9Kbw.jpg not found.
Image "C:\Users\LACHU\Downloads\yelp_photos\photos"/DeQqIXP4_u2F4nJ-THK82g.jpg not found.
Image "C:\Users\LACHU\Downloads\yelp_photos\photos"/Ideh4fp8DsfPBGV6hYVr-g.jpg not found.
Image "C:\Users\LACHU\Downloads\yelp_photos\photos"/YD-n2x0IHRs8GmUrBdoL7w.jpg not found.
Image "C:\Users\LACHU\Downloads\yelp_photos\photos"/Brt7JwkHzMl12NL1BBUQoA.jpg not found.
Image "C:\Users\LACHU\Downloads\yelp_photos\photos"/dY1Z8AetZH95W0maF4KX4A.jpg not found.
Image "C:\Users\LACHU\Downloads\yelp_photos\photo

In [ ]:
labels

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Building a CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compiling and training the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# After training, save the model
model.save('cnn_model.h5')


In [ ]:
from flask import Flask, request, jsonify
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from PIL import Image

app = Flask(__name__)

# Load the pre-trained CNN model
model = load_model('cnn_model.h5')

class_labels = ['class_1', 'class_2', 'class_3']

# Function to preprocess the image for prediction
def preprocess_image(img, target_size):
    img = img.resize(target_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    return img

# Prediction endpoint
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'})

    file = request.files['file']
    img = Image.open(file.stream)

    # Preprocess the image
    img_preprocessed = preprocess_image(img, target_size=(64, 64))
    predictions = model.predict(img_preprocessed)

    # For multi-class classification (softmax output)
    confidence_scores = predictions[0]
    predicted_class_index = np.argmax(confidence_scores)
    predicted_class_label = class_labels[predicted_class_index]

    confidence_dict = {label: float(confidence_scores[idx]) for idx, label in enumerate(class_labels)}
    return jsonify({
        'predicted_class': predicted_class_label,
        'confidence_scores': confidence_dict  # Confidence for all classes
    })

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Confidence Score


In [ ]:
{
  "predicted_class": "class_5",
  "confidence_scores": {
    "class_1": 0.1,
    "class_2": 0.85,
    "class_3": 0.05
  }
}


{'predicted_class': 'class_5',
 'confidence_scores': {'class_1': 0.1, 'class_2': 0.85, 'class_3': 0.05}}

In [ ]:
pip install flask keras pillow numpy tensorflow
